<a href="https://colab.research.google.com/github/wbs2788/yt-mt3/blob/master/y2b_2_MT3_in_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/mdnestor/yt-mt3/blob/master/MT3_for_noobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MT3 for Noobs

Instructions:

* **Step 1.** Run Chunk 1. This will install MT3 and its dependencies to Colab.
  * Note: Only run Chunk 1 once. If you mistakenly run this chunk twice, it results in an error.
      To remedy this, open the file menu in the upper left labeled "Runtime" and select "Factory reset runtime".

* **Step 2.** Prepare a csv document with some YouTube links in row. Then run Chunk 3.

* **Step 3.** Check urls read in Chunk 2, then run it.

You can re-run Step 2 & 3 as many times as you like. Enjoy!


In [ ]:
#@title Chunk 1: Setup.
# install yt-dlp
!python3 -m pip install --no-deps -U yt-dlp

# install stuff
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev

!pip install --upgrade pip
!pip install --upgrade jax
!pip install jaxlib -f https://storage.googleapis.com/jax-releases/jax_releases.html
!pip install nest-asyncio
!pip install pyfluidsynth

# install t5x
!git clone --branch=main https://github.com/google-research/t5x
!mv t5x t5x_tmp; mv t5x_tmp/* .; rm -r t5x_tmp
!sed -i 's:jax\[tpu\]:jax:' setup.py
!python3 -m pip install -e .

# install mt3
!git clone --branch=main https://github.com/magenta/mt3
!mv mt3 mt3_tmp; mv mt3_tmp/* .; rm -r mt3_tmp
!python3 -m pip install -e .

# copy checkpoints
!gsutil -q -m cp -r gs://mt3/checkpoints .

# copy soundfont (originally from https://sites.google.com/site/soundfonts4u)
!gsutil -q -m cp gs://magentadata/soundfonts/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2 .

#@title Imports and Definitions

import functools
import os

import numpy as np
import tensorflow.compat.v2 as tf

import functools
import gin
import jax
import librosa
import note_seq
import seqio
import t5
import t5x

from mt3 import metrics_utils
from mt3 import models
from mt3 import network
from mt3 import note_sequences
from mt3 import preprocessors
from mt3 import spectrograms
from mt3 import vocabularies

from google.colab import files

import nest_asyncio
nest_asyncio.apply()

SAMPLE_RATE = 16000
SF2_PATH = 'SGM-v2.01-Sal-Guit-Bass-V1.3.sf2'

def upload_audio(sample_rate):
  data = list(files.upload().values())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return note_seq.audio_io.wav_data_to_samples_librosa(
    data[0], sample_rate=sample_rate)



class InferenceModel(object):
  """Wrapper of T5X model for music transcription."""

  def __init__(self, checkpoint_path, model_type='mt3'):

    # Model Constants.
    if model_type == 'ismir2021':
      num_velocity_bins = 127
      self.encoding_spec = note_sequences.NoteEncodingSpec
      self.inputs_length = 512
    elif model_type == 'mt3':
      num_velocity_bins = 1
      self.encoding_spec = note_sequences.NoteEncodingWithTiesSpec
      self.inputs_length = 256
    else:
      raise ValueError('unknown model_type: %s' % model_type)

    gin_files = ['/content/mt3/gin/model.gin',
                 f'/content/mt3/gin/{model_type}.gin']

    self.batch_size = 8
    self.outputs_length = 1024
    self.sequence_length = {'inputs': self.inputs_length, 
                            'targets': self.outputs_length}

    self.partitioner = t5x.partitioning.PjitPartitioner(
        num_partitions=1)

    # Build Codecs and Vocabularies.
    self.spectrogram_config = spectrograms.SpectrogramConfig()
    self.codec = vocabularies.build_codec(
        vocab_config=vocabularies.VocabularyConfig(
            num_velocity_bins=num_velocity_bins))
    self.vocabulary = vocabularies.vocabulary_from_codec(self.codec)
    self.output_features = {
        'inputs': seqio.ContinuousFeature(dtype=tf.float32, rank=2),
        'targets': seqio.Feature(vocabulary=self.vocabulary),
    }

    # Create a T5X model.
    self._parse_gin(gin_files)
    self.model = self._load_model()

    # Restore from checkpoint.
    self.restore_from_checkpoint(checkpoint_path)

  @property
  def input_shapes(self):
    return {
          'encoder_input_tokens': (self.batch_size, self.inputs_length),
          'decoder_input_tokens': (self.batch_size, self.outputs_length)
    }

  def _parse_gin(self, gin_files):
    """Parse gin files used to train the model."""
    gin_bindings = [
        'from __gin__ import dynamic_registration',
        'from mt3 import vocabularies',
        'VOCAB_CONFIG=@vocabularies.VocabularyConfig()',
        'vocabularies.VocabularyConfig.num_velocity_bins=%NUM_VELOCITY_BINS'
    ]
    with gin.unlock_config():
      gin.parse_config_files_and_bindings(
          gin_files, gin_bindings, finalize_config=False)

  def _load_model(self):
    """Load up a T5X `Model` after parsing training gin config."""
    model_config = gin.get_configurable(network.T5Config)()
    module = network.Transformer(config=model_config)
    return models.ContinuousInputsEncoderDecoderModel(
        module=module,
        input_vocabulary=self.output_features['inputs'].vocabulary,
        output_vocabulary=self.output_features['targets'].vocabulary,
        optimizer_def=t5x.adafactor.Adafactor(decay_rate=0.8, step_offset=0),
        input_depth=spectrograms.input_depth(self.spectrogram_config))


  def restore_from_checkpoint(self, checkpoint_path):
    """Restore training state from checkpoint, resets self._predict_fn()."""
    train_state_initializer = t5x.utils.TrainStateInitializer(
      optimizer_def=self.model.optimizer_def,
      init_fn=self.model.get_initial_variables,
      input_shapes=self.input_shapes,
      partitioner=self.partitioner)

    restore_checkpoint_cfg = t5x.utils.RestoreCheckpointConfig(
        path=checkpoint_path, mode='specific', dtype='float32')

    train_state_axes = train_state_initializer.train_state_axes
    self._predict_fn = self._get_predict_fn(train_state_axes)
    self._train_state = train_state_initializer.from_checkpoint_or_scratch(
        [restore_checkpoint_cfg], init_rng=jax.random.PRNGKey(0))

  @functools.lru_cache()
  def _get_predict_fn(self, train_state_axes):
    """Generate a partitioned prediction function for decoding."""
    def partial_predict_fn(params, batch, decode_rng):
      return self.model.predict_batch_with_aux(
          params, batch, decoder_params={'decode_rng': None})
    return self.partitioner.partition(
        partial_predict_fn,
        in_axis_resources=(
            train_state_axes.params,
            t5x.partitioning.PartitionSpec('data',), None),
        out_axis_resources=t5x.partitioning.PartitionSpec('data',)
    )

  def predict_tokens(self, batch, seed=0):
    """Predict tokens from preprocessed dataset batch."""
    prediction, _ = self._predict_fn(
        self._train_state.params, batch, jax.random.PRNGKey(seed))
    return self.vocabulary.decode_tf(prediction).numpy()

  def __call__(self, audio):
    """Infer note sequence from audio samples.
    
    Args:
      audio: 1-d numpy array of audio samples (16kHz) for a single example.

    Returns:
      A note_sequence of the transcribed audio.
    """
    ds = self.audio_to_dataset(audio)
    ds = self.preprocess(ds)

    model_ds = self.model.FEATURE_CONVERTER_CLS(pack=False)(
        ds, task_feature_lengths=self.sequence_length)
    model_ds = model_ds.batch(self.batch_size)

    inferences = (tokens for batch in model_ds.as_numpy_iterator()
                  for tokens in self.predict_tokens(batch))

    predictions = []
    for example, tokens in zip(ds.as_numpy_iterator(), inferences):
      predictions.append(self.postprocess(tokens, example))

    result = metrics_utils.event_predictions_to_ns(
        predictions, codec=self.codec, encoding_spec=self.encoding_spec)
    return result['est_ns']

  def audio_to_dataset(self, audio):
    """Create a TF Dataset of spectrograms from input audio."""
    frames, frame_times = self._audio_to_frames(audio)
    return tf.data.Dataset.from_tensors({
        'inputs': frames,
        'input_times': frame_times,
    })

  def _audio_to_frames(self, audio):
    """Compute spectrogram frames from audio."""
    frame_size = self.spectrogram_config.hop_width
    padding = [0, frame_size - len(audio) % frame_size]
    audio = np.pad(audio, padding, mode='constant')
    frames = spectrograms.split_audio(audio, self.spectrogram_config)
    num_frames = len(audio) // frame_size
    times = np.arange(num_frames) / self.spectrogram_config.frames_per_second
    return frames, times

  def preprocess(self, ds):
    pp_chain = [
        functools.partial(
            t5.data.preprocessors.split_tokens_to_inputs_length,
            sequence_length=self.sequence_length,
            output_features=self.output_features,
            feature_key='inputs',
            additional_feature_keys=['input_times']),
        # Cache occurs here during training.
        preprocessors.add_dummy_targets,
        functools.partial(
            preprocessors.compute_spectrograms,
            spectrogram_config=self.spectrogram_config)
    ]
    for pp in pp_chain:
      ds = pp(ds)
    return ds

  def postprocess(self, tokens, example):
    tokens = self._trim_eos(tokens)
    start_time = example['input_times'][0]
    # Round down to nearest symbolic token step.
    start_time -= start_time % (1 / self.codec.steps_per_second)
    return {
        'est_tokens': tokens,
        'start_time': start_time,
        # Internal MT3 code expects raw inputs, not used here.
        'raw_inputs': []
    }

  @staticmethod
  def _trim_eos(tokens):
    tokens = np.array(tokens, np.int32)
    if vocabularies.DECODED_EOS_ID in tokens:
      tokens = tokens[:np.argmax(tokens == vocabularies.DECODED_EOS_ID)]
    return tokens


In [ ]:
#@title Chunk 2: Get Video List { vertical-output: true }
urls = []

def read_url_from_csv():
  data = list(files.upload().values())
  return str(data[0], encoding='utf-8') 
  
urls = read_url_from_csv()
urls = urls.split('\r\n')
urls = list(filter(None, urls))

print(len(urls))
print(urls)

In [ ]:
#@title Chunk 3: Transcription.

MODEL = "ismir2021" #@param ["mt3", "ismir2021"]
checkpoint_path = f'/content/checkpoints/{MODEL}/'
inference_model = InferenceModel(checkpoint_path, MODEL)

#!pip install moviepy

import moviepy.editor

def sampling(file_name, len=60):
  cnt = 0
  clip = moviepy.editor.VideoFileClip(file_name + ".webm")
  time = int(clip.duration)
  for j, i in enumerate(range(20, time - 100, 60)):
    if cnt > 20:
      break
    cnt += 1
    myclip = clip.subclip(i, i + 60)
    myclip.write_videofile(file_name+'({}).mp4'.format(str(j)),fps=20,
                    codec='mpeg4',bitrate='256k')
  return cnt

video_source = "YouTube" #@param ["YouTube", "Bilibili"]
tc = []
name = []

for i, audio_url in enumerate(urls):
  print(i / len(urls))
  audio_url = str(audio_url)
  output_name = audio_url.split("/")[-1]
  !yt-dlp {audio_url} -o {output_name} -q --force-overwrites
  video_name = output_name + ".webm"
  clip_num = sampling(output_name)
  os.remove(video_name)
  '''except BaseException:
    try: 
      video_name = output_name + ".mp4"
      with open(video_name, 'rb') as f:
        pass
      clip_num = sampling(output_name)
      os.remove(video_name)
    except BaseException:
      print("error:" + output_name)
      tc.append(None)
      continue'''
  for i in range(clip_num):
    with open(output_name+'({}).mp4'.format(str(i)), 'rb') as f:
      data = f.read()
    audio = note_seq.audio_io.wav_data_to_samples_librosa(data, sample_rate=SAMPLE_RATE)
    #note_seq.notebook_utils.colab_play(audio, sample_rate=SAMPLE_RATE)
    est_ns = inference_model(audio)
    tc.append(est_ns)
    name.append(output_name+'({})'.format(str(i)))
urls = name

In [ ]:
#@title Chunk 4(Option): Download MIDI File
start = 0 #@param 
for audio_url, est_ns in zip(urls[start:start + 10 ], tc[start:start + 10]):
  note_seq.sequence_proto_to_midi_file(est_ns, f'/tmp/{output_name}.mid')
  files.download(f'/tmp/{output_name}.mid')

In [ ]:
#@title Chunk 5: Preparation for Chord Extraction
!pip install miditoolkit
!pip install chorder

from typing import List, Tuple
import chorder
import miditoolkit as mtk


def extract_chord(midi_obj: mtk.midi.parser.MidiFile) -> List[Tuple[int, int, chorder.Chord]]:
    tick = midi_obj.ticks_per_beat
    chord_list = []
    ptr = 0
    last = chorder.Chord()
    l = len(chorder.Dechorder.dechord(midi_obj)) - 1
    for i, ch in enumerate(chorder.Dechorder.dechord(midi_obj)):
        if i == 0:
            continue
        if last == ch:
            if i == l:
                chord_list.append((ptr * tick, i * tick, ch))
            continue
        else:
            chord_list.append((ptr * tick, i * tick, last))
            last = ch
            ptr = i
    return chord_list

def print_chord(chord_list: List[Tuple[int, int, chorder.Chord]], file_name) -> None:
    with open(file_name, 'w') as f:
        for s, t, ch in chord_list:
            if ch.quality != None:
                f.write(str(s) + ' ' + str(t) + ' ' + str(ch) + '\n')
    
def execute_midi(file_path, file_name):
    midi_obj = mtk.midi.parser.MidiFile(file_path)
    print_chord(extract_chord(midi_obj)[1:], file_name)


In [ ]:
#@title Chunk 6(Option): Download Chord Extraction Result
new_start = 0#@param 
for audio_url, est_ns in zip(urls[new_start:new_start + 10], tc[start:start + 10]):
  note_seq.sequence_proto_to_midi_file(est_ns, f'/tmp/{output_name}.mid')
  execute_midi('/tmp/'+output_name+'.mid', output_name)
  files.download(output_name)

In [ ]:
#@title Chunk 7: Notality Analysis

In [ ]:
#@title Addition: Download Zip File
import zipfile
import os

zip_name = "scenery.zip" #@param
if os.path.exists(zip_name):
  os.remove(zip_name)

zipFilePath = os.path.join(zip_name)
zipFile = zipfile.ZipFile(zipFilePath, 'w', zipfile.ZIP_DEFLATED)
for audio_url, est_ns in zip(urls, tc):
  output_name = audio_url
  note_seq.sequence_proto_to_midi_file(est_ns, output_name+'.mid')
  # print("executing "+ output_name+'.mid')
  execute_midi(output_name+'.mid', output_name+'.txt')
  zipFile.write(output_name+'.mid')
  zipFile.write(output_name+'.txt')
  zipFile.write(output_name+'.mp4')
  os.remove(output_name+'.mid')
  os.remove(output_name+'.txt')
  os.remove(output_name+'.mp4')

files.download(zip_name)